In [44]:
import os
import requests
import pandas as pd
from time import sleep
from datetime import datetime, timedelta

#######################################
# You must insert your api key below. #
#######################################
API_KEY = "YOUR_API_KEY"

# Base Class


In [45]:
class VolumeData:
    unit = {
        ("spot", "all_exchange", "btc_usd"): "btc",
        ("spot", "binance", "btc_usdt"): "btc",
        ("spot", "binance_us", "btc_usd"): "btc",
        ("spot", "binance_us", "btc_usdt"): "btc",
        ("spot", "bitfinex", "btc_usd"): "btc",
        ("spot", "bitfinex", "btc_usdt"): "btc",
        ("spot", "bittrex", "btc_usd"): "usd",
        ("spot", "bittrex", "btc_usdt"): "usd",
        ("spot", "coinbase_pro", "btc_usd"): "btc",
        ("spot", "ftx", "btc_usd"): "usd",
        ("spot", "ftx", "btc_usdt"): "usd",
        ("spot", "gemini", "btc_usd"): "btc",
        ("spot", "huobi_global", "btc_usd"): "btc",
        ("spot", "kraken", "btc_usd"): "btc",
        ("spot", "kraken", "btc_usdt"): "btc",
        ("spot", "okex", "btc_usdt"): "btc",
        ("perpetual", "all_exchange", "btc_usd"): "btc",
        ("perpetual", "binance", "btc_usd"): "usd_100",
        ("perpetual", "binance", "btc_usdt"): "btc",
        ("perpetual", "bitmex", "btc_usd"): "usd",
        ("perpetual", "bybit", "btc_usd"): "usd",
        ("perpetual", "bybit", "btc_usdt"): "btc",
        ("perpetual", "deribit", "btc_usd"): "btc",
        ("perpetual", "ftx", "btc_usd"): "btc",
        ("perpetual", "huobi_global", "btc_usd"): "usd_100",
        ("perpetual", "huobi_global", "btc_usdt"): "btc",
        ("perpetual", "okex", "btc_usd"): "usd_100",
        ("perpetual", "okex", "btc_usdt"): "btc",
    }
    
    def __init__(self, coin, window, _from, _to):
        assert window in ["day", "hour", "min"]

        api_key = API_KEY
        self.coin = coin
        self.base_url = f"https://api.cryptoquant.com/v1/{self.coin}"
        self.data_path = f"./datasets_{window}/{self.coin}/"
        
        self.url_fn = lambda c, e: os.path.join(self.base_url, c, e)
        self.headers = {"Authorization": "Bearer " + api_key}
        self.window = window
        self.params = {
            "window": window,
            "from": _from,
            "to": _to,
            "limit": 100000,
        }
        self._from = _from
        self._to = _to

    def _get_data(self, c, e, params):
        if self.window == "day":
            idx = "date"
            idx_form = "%Y-%m-%d"
            td = timedelta(days=1)
        elif self.window == "hour":
            idx = "datetime"
            idx_form = "%Y-%m-%d %H:%M:%S"
            td = timedelta(hours=1)
        elif self.window == "min":
            idx = "datetime"
            idx_form = "%Y-%m-%d %H:%M:%S"
            td = timedelta(minutes=1)

        url = self.url_fn(c, e)
        _from = params["from"]
        _to = params["to"]

        records = []
        while _to > _from:
            _params = dict(params, **{"to": _to})
            resp = requests.get(url, params=_params, headers=self.headers)
            sleep(0.5)
            data = resp.json()
            if len(data["result"]["data"]) == 0:
                break
            last_date = datetime.strptime(data["result"]["data"][-1][idx], idx_form)
            _to = (last_date - td).strftime('%Y-%m-%dT%H:%M:%S')
            records += data["result"]["data"]

        datum = pd.DataFrame.from_records(records, index=idx)
        return datum
    
    def get_volume(self, market, exchange, symbol, unit, save_path=None):
        assert unit in ["btc", "usd"]
        
        category = "market-data"
        endpoint = "price-ohlcv"
        params = dict(self.params, **{"market": market, "exchange": exchange, "symbol": symbol})
        data = self._get_data(category, endpoint, params)[::-1]
        
        unit_info = self.unit[(market, exchange, symbol)]
        if len(unit_info.split("_")) == 1:
            native = unit_info
            cont_size = 1
        else:
            native = unit_info.split("_")[0]
            cont_size = int(unit_info.split("_")[1])
        
        if unit == "btc" and native == "usd":
            data["volume"] /= data["close"]
        elif unit == "usd" and native == "btc":
            data["volume"] *= data["close"]
        data["volume"] *= cont_size
        
        data = data["volume"]
        if save_path:
            os.makedirs(self.data_path, exist_ok=True)
            data.to_excel(os.path.join(self.data_path, save_path))
        return data

## Get Volume Data

## Define parameters

In [46]:
coin = "btc"  # base asset
window = "min"  # window: day, hour, min 
_from = "20210701T000000"  # from datetime
_to = "20210705T000000"  # to datetime
market = "spot"  # market of asset: spot, perpetual
exchange = "ftx"  # exchange
symbol = "btc_usdt"  # symbol
unit = "btc"  # volume unit

## Get data from CryptoQuant API and save data in excel

In [47]:
data = VolumeData(coin, window, _from, _to)
data.get_volume(market, exchange, symbol, unit, save_path="output.xlsx")

datetime
2021-07-01 00:00:00     2.555830
2021-07-01 00:01:00     6.281442
2021-07-01 00:02:00     9.718029
2021-07-01 00:03:00     1.793022
2021-07-01 00:04:00     0.512317
                         ...    
2021-07-04 23:56:00     7.252100
2021-07-04 23:57:00     0.243312
2021-07-04 23:58:00     4.407304
2021-07-04 23:59:00     0.569475
2021-07-05 00:00:00    21.781438
Name: volume, Length: 5761, dtype: float64